<a href="https://colab.research.google.com/github/Marcopoloaltieri/Objtos-Intelgentes/blob/main/SX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq>/dev/null


In [13]:
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz

In [16]:
!tar xf /content/spark-3.4.4-bin-hadoop3.tgz

In [22]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

In [18]:
!pip install -q findspark

In [23]:
import findspark
findspark.init()

In [61]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

In [25]:
#Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [26]:
# Inicializando a sessão Spark
spark = SparkSession.builder \
    .appName("SX_ENEM_2020") \
    .getOrCreate()

In [47]:
# Extração: leitura do arquivo CSV
# Neste caso tambem há a opção do google.collabe open file que importa o arquivo para dentro do código partindo da sua máquina porém devido ao volume de dados ele é demorado e compensa mais subir o arquivo direto e usar o comando .load(/content/)
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ";") \
    .load("/content/MICRODADOS_ENEM_2020.csv")

In [50]:
# Exibindo o Scheme (tipos das variaveis)
df.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [49]:
# Exibindo o DataFrame
print("DataFrame")
df.show()

DataFrame
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_

In [56]:
#Filtrar colunas que eu quero trabalhar
df_fato = df.select('NU_INSCRICAO','NU_ANO','TP_FAIXA_ETARIA','TP_SEXO','TP_COR_RACA','TP_ESCOLA','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT')

In [57]:
#Mostra df fato
df_fato.show()

+------------+------+---------------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_ESCOLA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|
+------------+------+---------------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+
|200006271946|  2020|             11|      F|          2|        1|             0|             0|             0|             0|      null|      null|      null|      null|
|200001195856|  2020|             11|      M|          3|        1|             1|             1|             1|             1|     604.1|     661.7|     595.3|     711.3|
|200001943954|  2020|              4|      F|          3|        2|             0|             0|             0|             0|      null|  

In [62]:
# Criando  dimensão para faixa etária
df_faixa_data = [
    (1, "Menor de 17 anos"),
    (2, "17 anos"),
    (3, "18 anos"),
    (4, "19 anos"),
    (5, "20 anos"),
    (6, "21 anos"),
    (7, "22 anos"),
    (8, "23 anos"),
    (9, "24 anos"),
    (10, "25 anos"),
    (11, "Entre 26 e 30 anos"),
    (12, "Entre 31 e 35 anos"),
    (13, "Entre 36 e 40 anos"),
    (14, "Entre 41 e 45 anos"),
    (15, "Entre 46 e 50 anos"),
    (16, "Entre 51 e 55 anos"),
    (17, "Entre 56 e 60 anos"),
    (18, "Entre 61 e 65 anos"),
    (19, "Entre 66 e 70 anos"),
    (20, "Maior de 70 anos")
]

schema_faixa = StructType([
    StructField("TP_FAIXA_ETARIA", IntegerType(), True),
    StructField("DESC_FAIXA_ETARIA", StringType(), True)
])

In [64]:
df_faixa_etaria = spark.createDataFrame(df_faixa_data, schema=schema_faixa)

In [65]:
# Exibindo a dimensão de faixa etária
print("Dimensão: Faixa Etária")
df_faixa_etaria.show()

Dimensão: Faixa Etária
+---------------+------------------+
|TP_FAIXA_ETARIA| DESC_FAIXA_ETARIA|
+---------------+------------------+
|              1|  Menor de 17 anos|
|              2|           17 anos|
|              3|           18 anos|
|              4|           19 anos|
|              5|           20 anos|
|              6|           21 anos|
|              7|           22 anos|
|              8|           23 anos|
|              9|           24 anos|
|             10|           25 anos|
|             11|Entre 26 e 30 anos|
|             12|Entre 31 e 35 anos|
|             13|Entre 36 e 40 anos|
|             14|Entre 41 e 45 anos|
|             15|Entre 46 e 50 anos|
|             16|Entre 51 e 55 anos|
|             17|Entre 56 e 60 anos|
|             18|Entre 61 e 65 anos|
|             19|Entre 66 e 70 anos|
|             20|  Maior de 70 anos|
+---------------+------------------+



In [66]:
# Realizando o join entre o fato e a dimensão
df_fato_com_dim = df_fato.join(df_faixa_etaria, on="TP_FAIXA_ETARIA", how="left")

In [67]:
print("Fato com dimensão de Faixa Etária")
df_fato_com_dim.show()

Fato com dimensão de Faixa Etária
+---------------+------------+------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+------------------+
|TP_FAIXA_ETARIA|NU_INSCRICAO|NU_ANO|TP_SEXO|TP_COR_RACA|TP_ESCOLA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT| DESC_FAIXA_ETARIA|
+---------------+------------+------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+------------------+
|             13|200005966328|  2020|      M|          2|        1|             1|             1|             1|             1|     486.0|     503.3|     439.8|     405.6|Entre 36 e 40 anos|
|              6|200004902048|  2020|      M|          1|        1|             0|             0|             0|             0|      null|      null|      null|      null|           21 anos|
|          

In [68]:
# Criação do DataFrame de dimensão para TP_SEXO
sexo_data = [
    ("M", "Masculino"),
    ("F", "Feminino")
]

schema_sexo = StructType([
    StructField("TP_SEXO", StringType(), True),
    StructField("DESC_SEXO", StringType(), True)
])

df_sexo = spark.createDataFrame(sexo_data, schema=schema_sexo)


In [69]:
# Realizando o join entre o fato e a dimensão de TP_SEXO
df_fato_com_sexo = df_fato.join(df_sexo, on="TP_SEXO", how="left")

In [70]:
print("Fato com dimensão de TP_SEXO")
df_fato_com_sexo.show()

Fato com dimensão de TP_SEXO
+-------+------------+------+---------------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+
|TP_SEXO|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_COR_RACA|TP_ESCOLA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|DESC_SEXO|
+-------+------------+------+---------------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+
|      F|200006271946|  2020|             11|          2|        1|             0|             0|             0|             0|      null|      null|      null|      null| Feminino|
|      F|200001943954|  2020|              4|          3|        2|             0|             0|             0|             0|      null|      null|      null|      null| Feminino|
|      F|200001634757|  2020|              4|          3|    

In [71]:
# Criação do DataFrame de dimensão para TP_COR_RACA
cor_raca_data = [
    (0, "Não declarado"),
    (1, "Branca"),
    (2, "Preta"),
    (3, "Parda"),
    (4, "Amarela"),
    (5, "Indígena")
]

schema_cor_raca = StructType([
    StructField("TP_COR_RACA", IntegerType(), True),
    StructField("DESC_COR_RACA", StringType(), True)
])

df_cor_raca = spark.createDataFrame(cor_raca_data, schema=schema_cor_raca)


In [72]:
# Realizando o join entre o DataFrame fato e a dimensão de TP_COR_RACA
df_fato_com_cor_raca = df_fato.join(df_cor_raca, on="TP_COR_RACA", how="left")

# Exibindo o DataFrame fato enriquecido com a descrição da cor/raça
print("Fato com dimensão de TP_COR_RACA")
df_fato_com_cor_raca.show()

Fato com dimensão de TP_COR_RACA
+-----------+------------+------+---------------+-------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+-------------+
|TP_COR_RACA|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESCOLA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|DESC_COR_RACA|
+-----------+------------+------+---------------+-------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+-------------+
|          1|200001334237|  2020|              4|      M|        1|             1|             1|             1|             1|     604.6|     604.8|     562.1|     753.2|       Branca|
|          1|200004902048|  2020|              6|      M|        1|             0|             0|             0|             0|      null|      null|      null|      null|       Branca|
|          1|200006138472|  2020|    

In [74]:
presenca_data = [
    (0, "Faltou à prova"),
    (1, "Presente na prova"),
    (2, "Eliminado na prova")
]
schema_presenca = StructType([
    StructField("TP_PRESENCA", IntegerType(), True),
    StructField("DESC_PRESENCA", StringType(), True)
])
df_presenca = spark.createDataFrame(presenca_data, schema=schema_presenca)


In [75]:
# Cada coluna de presença, possui uma versão apontanddo para dimensao
df_presenca_cn = df_presenca \
    .withColumnRenamed("TP_PRESENCA", "TP_PRESENCA_CN") \
    .withColumnRenamed("DESC_PRESENCA", "DESC_PRESENCA_CN")

df_presenca_ch = df_presenca \
    .withColumnRenamed("TP_PRESENCA", "TP_PRESENCA_CH") \
    .withColumnRenamed("DESC_PRESENCA", "DESC_PRESENCA_CH")

df_presenca_lc = df_presenca \
    .withColumnRenamed("TP_PRESENCA", "TP_PRESENCA_LC") \
    .withColumnRenamed("DESC_PRESENCA", "DESC_PRESENCA_LC")

df_presenca_mt = df_presenca \
    .withColumnRenamed("TP_PRESENCA", "TP_PRESENCA_MT") \
    .withColumnRenamed("DESC_PRESENCA", "DESC_PRESENCA_MT")

# Realizando os joins para adicionar a descrição de cada coluna
df_fato_enriched = df_fato \
    .join(df_presenca_cn, df_fato["TP_PRESENCA_CN"] == df_presenca_cn["TP_PRESENCA_CN"], "left") \
    .join(df_presenca_ch, df_fato["TP_PRESENCA_CH"] == df_presenca_ch["TP_PRESENCA_CH"], "left") \
    .join(df_presenca_lc, df_fato["TP_PRESENCA_LC"] == df_presenca_lc["TP_PRESENCA_LC"], "left") \
    .join(df_presenca_mt, df_fato["TP_PRESENCA_MT"] == df_presenca_mt["TP_PRESENCA_MT"], "left")


In [76]:
# Exibindo o DataFrame fato enriquecido com as descrições de presença
df_fato_enriched.show(truncate=False)

+------------+------+---------------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+--------------+-----------------+--------------+-----------------+--------------+-----------------+--------------+-----------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_ESCOLA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|TP_PRESENCA_CN|DESC_PRESENCA_CN |TP_PRESENCA_CH|DESC_PRESENCA_CH |TP_PRESENCA_LC|DESC_PRESENCA_LC |TP_PRESENCA_MT|DESC_PRESENCA_MT |
+------------+------+---------------+-------+-----------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+--------------+-----------------+--------------+-----------------+--------------+-----------------+--------------+-----------------+
|200001195856|2020  |11             |M      |3          |1        |1             |1     

In [82]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [83]:
# Salvando o DataFrame Fato no Google Drive
df_fato.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Fato/df_fato.csv")

# Salvando as dimensões
df_faixa_etaria.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Faixa_etaria/df_faixa_etaria.csv")

df_sexo.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Sexo/df_sexo.csv")

df_cor_raca.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Raça/df_cor_raca.csv")

df_presenca.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Fato - Presença/df_presenca.csv")


In [84]:
# Para o DataFrame Fato
df_fato.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Fato/df_fato_single")

# Para a dimensão Faixa Etária
df_faixa_etaria.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Faixa_etaria/df_faixa_etaria_single")

# Para a dimensão Sexo
df_sexo.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Sexo/df_sexo_single")

# Para a dimensão Cor/Raça
df_cor_raca.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Raça/df_cor_raca_single")

# Para a dimensão Presença
df_presenca.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///content/drive/MyDrive/SX/dM_Fato - Presença/df_presenca_single")


In [80]:
# Exportando o DataFrame Fato
df_fato.write.mode("overwrite") \
    .option("header", "true") \
    .csv("file:///C:/Users/marco/OneDrive/Área de Trabalho/SX/dM_Fato/df_fato.csv")

# Exportando as dimensões
df_faixa_etaria.write.mode("overwrite") \
    .option("header", "true") \
    .csv(r"file:///C:\Users\marco\OneDrive\Área de Trabalho\SX\dM_Faixa_etaria\df_faixa_etaria.csv")

df_sexo.write.mode("overwrite") \
    .option("header", "true") \
    .csv(r"file:///C:\Users\marco\OneDrive\Área de Trabalho\SX\dM_Sexo\df_sexo.csv")

df_cor_raca.write.mode("overwrite") \
    .option("header", "true") \
    .csv(r"file:///C:\Users\marco\OneDrive\Área de Trabalho\SX\dM_Raça\df_cor_raca.csv")

df_presenca.write.mode("overwrite") \
    .option("header", "true") \
    .csv(r"file:///C:\Users\marco\OneDrive\Área de Trabalho\SX\dM_Fato - Presença\df_presenca.csv")


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.4.4-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.4.4-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 